In [1]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

import cv2

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np


In [22]:
import cv2
import mediapipe as mp
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

hand_landmarks_dict = {
    0: "WRIST",
    1: "THUMB_CMC",
    2: "THUMB_MCP",
    3: "THUMB_IP",
    4: "THUMB_TIP",
    5: "INDEX_FINGER_MCP",
    6: "INDEX_FINGER_PIP",
    7: "INDEX_FINGER_DIP",
    8: "INDEX_FINGER_TIP",
    9: "MIDDLE_FINGER_MCP",
    10: "MIDDLE_FINGER_PIP",
    11: "MIDDLE_FINGER_DIP",
    12: "MIDDLE_FINGER_TIP",
    13: "RING_FINGER_MCP",
    14: "RING_FINGER_PIP",
    15: "RING_FINGER_DIP",
    16: "RING_FINGER_TIP",
    17: "PINKY_MCP",
    18: "PINKY_PIP",
    19: "PINKY_DIP",
    20: "PINKY_TIP"
}

# 손 랜드마크 추출 함수
def get_hand_landmarks(frame, hands):
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(image_rgb)
    if result.multi_hand_landmarks:
        landmarks = []
        for hand_landmarks in result.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                landmarks.append([lm.x, lm.y, lm.z])
        return np.array(landmarks)
    return []




In [35]:
import cv2
import mediapipe as mp
import numpy as np

# 손 랜드마크 번호를 미리 정의
hand_landmarks_dict = {
    0: "WRIST",
    1: "THUMB_CMC",
    2: "THUMB_MCP",
    3: "THUMB_IP",
    4: "THUMB_TIP",
    5: "INDEX_FINGER_MCP",
    6: "INDEX_FINGER_PIP",
    7: "INDEX_FINGER_DIP",
    8: "INDEX_FINGER_TIP",
    9: "MIDDLE_FINGER_MCP",
    10: "MIDDLE_FINGER_PIP",
    11: "MIDDLE_FINGER_DIP",
    12: "MIDDLE_FINGER_TIP",
    13: "RING_FINGER_MCP",
    14: "RING_FINGER_PIP",
    15: "RING_FINGER_DIP",
    16: "RING_FINGER_TIP",
    17: "PINKY_MCP",
    18: "PINKY_PIP",
    19: "PINKY_DIP",
    20: "PINKY_TIP"
}

# 손 랜드마크 추출 함수
def get_hand_landmarks(frame, hands):
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(image_rgb)
    if result.multi_hand_landmarks:
        landmarks = []
        for hand_landmarks in result.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                landmarks.append([lm.x, lm.y, lm.z])
        return np.array(landmarks)
    return []

# Cosine Similarity 계산 함수
def calculate_similarity(hand1, hand2):
    if hand1 is not None and hand2 is not None:
        # 유사도 계산
        hand1_flat = hand1.flatten().reshape(1, -1)
        hand2_flat = hand2.flatten().reshape(1, -1)
        similarity = cosine_similarity(hand1_flat, hand2_flat)[0][0]
        return similarity
    return 0

# 두 벡터 간의 각도 계산 함수
def calculate_angle(v1, v2):
    dot_product = np.dot(v1, v2)
    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)
    cos_angle = dot_product / (magnitude_v1 * magnitude_v2)
    angle = np.arccos(cos_angle)
    return np.degrees(angle)

# 동작 감지, 각도 측정 함수
def detect_hand_interactions(landmarks, angle_threshold=80, angle_range=10): # 실험해보고 정함 (90도가 이상적인데, 해보니까 잘 안나옴) # 80이하는 정확도가 너무 낮아짐
    if len(landmarks) == 21:  # 한 손이 감지된 경우
        wrist = landmarks[0]
        thumb_tip = landmarks[4]
        index_tip = landmarks[8]
        
        # 벡터 계산
        v1 = np.array(thumb_tip) - np.array(wrist)
        v2 = np.array(index_tip) - np.array(wrist)
        
        # 두 벡터 간의 각도 계산
        angle = calculate_angle(v1, v2)
        
        if angle_threshold <= angle <= (angle_threshold + angle_range):
            return angle, True
        else:
            return angle, False
    return None, False

In [8]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 2.3 MB/s eta 0:00:0000:0100:01


In [3]:
!pip install scikit-learn

  Using cached scikit_learn-1.5.1-cp39-cp39-macosx_10_9_x86_64.whl.metadata (12 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.5.1-cp39-cp39-macosx_10_9_x86_64.whl (12.1 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


### 타겟 비디오의 손 관절 측정


In [36]:
# 색상 팔레트
colors={
    "purple_1":[120, 58, 221],
    "purple_2":[147, 97, 228],
    "purple_3":[174, 137, 235],
    "purple_4":[201, 176, 241],
    "purple_5":[228, 216, 248]
}

### test code

In [37]:
# 비디오 파일 경로
video_path = "../demo.mp4"  # 여기에 비디오 파일 경로를 입력하세요
cap = cv2.VideoCapture(video_path)

# 첫 번째 프레임 읽기
ret, first_frame = cap.read()
mp_hands = mp.solutions.hands
target_rgb = cv2.cvtColor(first_frame, cv2.COLOR_BGR2RGB)
target_result = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7, min_tracking_confidence=0.7).process(target_rgb)

I0000 00:00:1724223907.044541 2429755 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1 Pro
W0000 00:00:1724223907.058501 2926766 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1724223907.078361 2926760 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [38]:
target_result.multi_hand_landmarks[:3]

[landmark {
   x: 0.735671878
   y: 0.438989729
   z: 1.81169199e-07
 }
 landmark {
   x: 0.689108193
   y: 0.370228887
   z: -0.00946707651
 }
 landmark {
   x: 0.634040654
   y: 0.351166785
   z: -0.0173024926
 }
 landmark {
   x: 0.586739719
   y: 0.326370806
   z: -0.0223623514
 }
 landmark {
   x: 0.559816122
   y: 0.274949491
   z: -0.0279654562
 }
 landmark {
   x: 0.614430904
   y: 0.442905277
   z: -0.0283782203
 }
 landmark {
   x: 0.551114798
   y: 0.448273182
   z: -0.0340142436
 }
 landmark {
   x: 0.514758587
   y: 0.443601102
   z: -0.037010204
 }
 landmark {
   x: 0.486967474
   y: 0.443099797
   z: -0.039922554
 }
 landmark {
   x: 0.621275544
   y: 0.505222678
   z: -0.0236355215
 }
 landmark {
   x: 0.572940767
   y: 0.486672
   z: -0.0236896481
 }
 landmark {
   x: 0.582831323
   y: 0.452062309
   z: -0.0171936732
 }
 landmark {
   x: 0.601314425
   y: 0.438074529
   z: -0.0143165588
 }
 landmark {
   x: 0.634251773
   y: 0.549080908
   z: -0.0177848656
 }
 landmark

# mediapipe

In [40]:
import cv2
import mediapipe as mp
import time

# Mediapipe 손 모델 초기화
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# 색상 및 두께 지정
landmark_drawing_spec = mp_drawing.DrawingSpec(color=colors["purple_2"], thickness=2) 
connection_drawing_spec = mp_drawing.DrawingSpec(color=colors["purple_1"], thickness=2) 


# 비디오 파일 경로
video_path = "../demo.mp4"  # 여기에 비디오 파일 경로를 입력하세요
target_cap = cv2.VideoCapture(video_path)

web_cap = cv2.VideoCapture(0)
# 첫 번째 프레임 읽기
ret, first_frame = target_cap.read()

# 동작 및 각도 측정을 위한 변수 초기화
movement_count = 0
angle_threshold = 80  # 각도 임계값
angle_range = 10  # 각도 범위

start_time = time.time()
while(True):
    # 해상도 설정
    frame_width = int(web_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(web_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = web_cap.get(cv2.CAP_PROP_FPS)

    print(f'현재 해상도: {frame_width}x{frame_height}', f'현재 fps:{fps}')

    web_cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)  # 가로 해상도
    web_cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 360)  # 세로 해상도

    # 프레임 속도 설정 (예: 30 FPS)
    web_cap.set(cv2.CAP_PROP_FPS, 30)  # 초당 프레임 수
    
    with mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands:
        while web_cap.isOpened():
            target_ret, target_frame = target_cap.read()
            web_ret, web_frame = web_cap.read()
            if not web_ret:
                break
            if not target_ret:
                break

            # 타겟 손 랜드마크 추출
            target_rgb = cv2.cvtColor(target_frame, cv2.COLOR_BGR2RGB)
            web_rgb = cv2.cvtColor(web_frame, cv2.COLOR_BGR2RGB)
            target_result = hands.process(target_rgb)
            web_result = hands.process(web_rgb)
            
            current_time = time.time()
                 
            # 재활자의 랜드마크 표시
            if web_result.multi_hand_landmarks:
                for web_landmarks in web_result.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(web_frame, web_landmarks, mp_hands.HAND_CONNECTIONS,
                                            landmark_drawing_spec,
                                            connection_drawing_spec)
                    
                    # 동작 및 각도 감지
                    landmarks = get_hand_landmarks(web_frame, hands)
                    angle, detected = detect_hand_interactions(landmarks)
                    
                    if detected:
                        global interaction_detected, movement_count
                        if not interaction_detected:
                            interaction_detected = True
                            movement_count += 1
                    else:
                        interaction_detected = False
                    
                    if angle is not None:
                        cv2.putText(web_frame, f'Angle: {angle:.2f}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                    
            
            # # 타겟 랜드마크 표시
            # if target_result.multi_hand_landmarks:
            #     for target_landmarks in target_result.multi_hand_landmarks:
            #         # mp_drawing.draw_landmarks(target_frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            #         mp_drawing.draw_landmarks(web_frame, target_landmarks, mp_hands.HAND_CONNECTIONS)
                    
                    
            # 운동 시간 출력   
            elapsed_time = time.time() - start_time
            
            # 경과 시간을 분:초 형식으로 변환
            hours = int(elapsed_time//3600)
            minutes = int(elapsed_time // 60)
            seconds = int(elapsed_time % 60)
            time_text = f'Time: :{minutes:02}:{seconds:02}'     
            cv2.putText(web_frame, time_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            # 동작 횟수 출력
            cv2.putText(web_frame, f'Movement Count: {movement_count}', (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            # 화면에 프레임 표시
            cv2.imshow('Hand Tracking', web_frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                target_cap.release()
                cv2.destroyAllWindows()
                exit()
    target_cap.release()
cv2.destroyAllWindows()


end_time=time.time()

현재 해상도: 1920x1080 현재 fps:30.0


I0000 00:00:1724224568.839961 2429755 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1 Pro
W0000 00:00:1724224568.849600 3296062 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1724224568.857050 3296062 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


KeyboardInterrupt: 

### 입력된 비디오의 두손 유사도

In [ ]:
# Mediapipe 손 모델 초기화
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# 비디오 캡처 초기화
target_cap = cv2.VideoCapture(0)

with mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands:
    while target_cap.isOpened():
        target_ret, target_frame = target_cap.read()
        if not target_ret:
            break

        # 손 랜드마크 추출
        hand_landmarks = get_hand_landmarks(target_frame, hands)

        # 손 유사도 계산 (두 손이 모두 인식된 경우)
        if hand_landmarks is not None and len(hand_landmarks) == 42:  # 두 손 모두 21개 랜드마크씩 총 42개
            hand1 = hand_landmarks[:21]
            hand2 = hand_landmarks[21:]
            similarity = calculate_similarity(hand1, hand2)
            cv2.putText(target_frame, f'Similarity: {similarity:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        # 랜드마크 시각화
        # if hand_landmarks is not None:
        #     for hand_landmark in hand_landmarks.reshape(-1, 21, 3):
        #         mp_drawing.draw_landmarks(frame, mp_hands.HandLandmarkList(landmark=hand_landmark))

        cv2.imshow('Hand Tracking', target_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

target_cap.release()
cv2.destroyAllWindows()

I0000 00:00:1724215070.474400       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1


### 입력된 비디오의 동작 횟수 측정